In [ ]:
#importing the libraries required for the project
import os
import subprocess

VEP_DIR = os.path.expanduser("/full/path/to/vep_data") # add here your full path to vep_data
TMP_DIR = os.path.join(VEP_DIR, "tmp")
os.makedirs(TMP_DIR, exist_ok=True)

FASTA_FILE = "/full/path/to/GRCh38.fa" # add here your full path to GRCh38.fa
ASSEMBLY = "GRCh38"
INPUT_VCF = "/full/path/to/input.vcf" # add here your full path to input vcf file
PLUGINS_DIR = "/full/path/to/Plugins" # add here your full path to Plugins directory
REVEL_FILE = f"{PLUGINS_DIR}/new_tabbed_revel_grch38.tsv.gz"
MAXENTSCAN_FILE = f"{PLUGINS_DIR}/fordownload"
SPLICEAI_SNV = f"{PLUGINS_DIR}/spliceai_scores.raw.snv.hg38.vcf.gz"
SPLICEAI_INDEL = f"{PLUGINS_DIR}/spliceai_scores.raw.indel.hg38.vcf.gz"
SPLICEVAULT_FILE = f"{PLUGINS_DIR}/SpliceVault_data_GRCh38.tsv.gz"
ALPHAMISSENSE_FILE = f"{PLUGINS_DIR}/AlphaMissense_hg38.tsv.gz"
LOFTEE_PATH = f"{PLUGINS_DIR}/loftee"
HUMAN_ANCESTOR_FA = f"{LOFTEE_PATH}/human_ancestor.fa"
CONSERVATION_FILE = f"{LOFTEE_PATH}/phylocsf_gerp.sql"
PANGOLIN_SCRIPT = f"{PLUGINS_DIR}/Pangolin/scripts/gencode.v38.annotation.db"

def run_vep_with_plugins():
    output_file = f"{VEP_DIR}/tmp/output1.vcf"
    command = [
        "singularity", "exec", "vep.sif", "vep",
        "--dir", VEP_DIR,
        "--cache", "--offline",
        "--fasta", FASTA_FILE,
        "--assembly", ASSEMBLY,
        "--everything", "--format", "vcf", "--vcf", "--force_overwrite",
        "-i", INPUT_VCF, "-o", output_file,
        "--plugin", f"REVEL,file={REVEL_FILE}",
        "--plugin", f"MaxEntScan,{MAXENTSCAN_FILE}",
        "--plugin", f"SpliceAI,snv={SPLICEAI_SNV},indel={SPLICEAI_INDEL}",
        "--plugin", f"SpliceVault,file={SPLICEVAULT_FILE}",
        "--plugin", f"AlphaMissense,file={ALPHAMISSENSE_FILE}"
    ]
    subprocess.run(command, check=True)

def extract_fields_with_bcftools(input_file, output_file, fields):
    command = [
        "/full/path/to/bcftools", "+split-vep",  # add here your full path to bcftools
        input_file, "-f", fields, "-d",
        "-o", output_file
    ]
    subprocess.run(" ".join(command), shell=True, check=True)

def run_vep_with_loftee():
    output_file = f"{VEP_DIR}/tmp/output2.vcf"
    command = [
        "singularity", "exec", "vep.sif", "vep",
        "--dir", VEP_DIR,
        "--cache", "--offline",
        "--fasta", FASTA_FILE,
        "--assembly", ASSEMBLY,
        "--everything", "--format", "vcf", "--vcf", "--force_overwrite",
        "-i", INPUT_VCF, "-o", output_file,
        "--plugin", f"LoF,loftee_path:{LOFTEE_PATH},human_ancestor_fa:{HUMAN_ANCESTOR_FA},conservation_file:{CONSERVATION_FILE}",
        "--dir_plugins", LOFTEE_PATH
    ]
    subprocess.run(command, check=True)

    extract_fields_with_bcftools(
        input_file=output_file,
        output_file=f"{VEP_DIR}/tmp/splitted_output2.csv",
        fields="'%Feature %LoF\n'"
    )

def run_bcftools_query(input_file, output_dir, fields):
    output_file = output_dir + "splitted_output3.vcf"
    command = ["/full/path/to/bcftools", "query", # add here your full path to bcftools
               "-f", fields, input_file, "-o", output_file]
    subprocess.run(" ".join(command), shell=True, check=True)
    
def run_pangolin():
    # Runs Pangolin for annotation
    output_file = f"{VEP_DIR}/tmp/pangolin_output"
    command = [
        "/full/path/to/pangolin", INPUT_VCF, # add here your full path to pangolin
        "/full/path/to/GRCh38.primary_assembly.genome.fa.gz", # add here your full path to GRCh38.primary_assembly.genome.fa.gz file
        PANGOLIN_SCRIPT, output_file
    ]
    subprocess.run(command, check=True)

    run_bcftools_query(
        input_file = output_file + ".vcf",
        output_dir = f"{VEP_DIR}/tmp/",
        fields = "'%POS %REF %ALT %INFO\n'"
    )

# Main execution
run_vep_with_plugins()
extract_fields_with_bcftools(
    input_file=f"{VEP_DIR}/tmp/output1.vcf",
    output_file=f"{VEP_DIR}/tmp/splitted_output.csv",
    fields="'%CHROM %POS %REF %ALT %Existing_variation %SYMBOL %Gene %DOMAINS %Feature %MANE_SELECT %Consequence %IMPACT %HGVSc %HGVSp %CLIN_SIG %EXON %INTRON %REVEL %MaxEntScan_alt %MaxEntScan_diff %MaxEntScan_ref %SpliceAI_pred_DP_AG %SpliceAI_pred_DP_AL %SpliceAI_pred_DP_DG %SpliceAI_pred_DP_DL %SpliceAI_pred_DS_AG %SpliceAI_pred_DS_AL %SpliceAI_pred_DS_DG %SpliceAI_pred_DS_DL %SpliceVault_site_type %SpliceVault_top_events %am_pathogenicity\n'"
)

run_vep_with_loftee()
run_pangolin()

In [ ]:
#importing the libraries required for the project
import pandas as pd
import numpy as np
import math

column_names=["chromosome", "Locus (hg38)", "Ref", "Alt", "Variant ID", "Gene", "Gene ID", "DOMAIN", "Transcript ID", "Mane Select", "Variant type", "Impact", "HGVSc", "HGVSp", "ClinVar interpretation", "Exon", "Intron", "REVEL", "MaxEntScan Alt", "MaxEntScan Diff", "MaxEntScan Ref", "SpliceAI AG position", "SpliceAI AL position", "SpliceAI DG position", "SpliceAI DL position", "SpliceAI AG score", "SpliceAI AL score", "SpliceAI DG score", "SpliceAI DL score", "SpliceVault site type", "SpliceVault top events", "AlphaMissense"]
df = pd.read_csv(f'{TMP_DIR}/splitted_output.csv',sep=' ',names=column_names)

new_column_1 = df["Variant type"].str.split("&", n=4, expand=True)
df.insert(5,'Variant Type',new_column_1[0])
new_column_2 = df["Variant ID"].str.split("&", n=8, expand=True)
df.insert(4,'Variant Id',new_column_2[0])
new_column_3 = df["SpliceVault top events"].str.split("&", n=3, expand=True)
df.insert(29,'SpliceVault Top Event',new_column_3[0])
new_column_4 = df["Exon"].str.split("/", n=2, expand=True)
df.insert(15,'EXON',new_column_4[0])
df.insert(16,'EXON TOT',new_column_4[1])
new_column_5 = df["Intron"].str.split("/", n=2, expand=True)
df.insert(17,'INTRON',new_column_5[0])
df.insert(18,'INTRON TOT',new_column_5[1])
new_column_6 = df["DOMAIN"].str.split("Pfam:", n=2, expand=True)
df.insert(7,'PFAM',new_column_6[1])
new_column_7 = df["PFAM"].str.split("&", n=8, expand=True)
df.insert(9,'PFAM_ACCESSION',new_column_7[0])

del df["Variant type"]
del df["Variant ID"]
del df["SpliceVault top events"]
del df["Exon"]
del df["Intron"]
del df["PFAM"]
del df["DOMAIN"]

df['MaxEntScan Ref'] = pd.to_numeric(df['MaxEntScan Ref'], errors='coerce')
df['MaxEntScan Alt'] = pd.to_numeric(df['MaxEntScan Alt'], errors='coerce')
df['MaxEntScan Diff'] = pd.to_numeric(df['MaxEntScan Diff'], errors='coerce')
df['REVEL'] = pd.to_numeric(df['REVEL'], errors='coerce')
df['SpliceAI AG score'] = pd.to_numeric(df['SpliceAI AG score'], errors='coerce')
df['SpliceAI AL score'] = pd.to_numeric(df['SpliceAI AL score'], errors='coerce')
df['SpliceAI DG score'] = pd.to_numeric(df['SpliceAI DG score'], errors='coerce')
df['SpliceAI DL score'] = pd.to_numeric(df['SpliceAI DL score'], errors='coerce')
df['SpliceAI AG position'] = pd.to_numeric(df['SpliceAI AG position'], errors='coerce')
df['SpliceAI AL position'] = pd.to_numeric(df['SpliceAI AL position'], errors='coerce')
df['SpliceAI DG position'] = pd.to_numeric(df['SpliceAI DG position'], errors='coerce')
df['SpliceAI DL position'] = pd.to_numeric(df['SpliceAI DL position'], errors='coerce')
df['INTRON'] = pd.to_numeric(df['INTRON'], errors='coerce')
df['INTRON TOT'] = pd.to_numeric(df['INTRON TOT'], errors='coerce')
df['EXON'] = pd.to_numeric(df['EXON'], errors='coerce')
df['EXON TOT'] = pd.to_numeric(df['EXON TOT'], errors='coerce')
df['AlphaMissense'] = pd.to_numeric(df['AlphaMissense'], errors='coerce')

df['REVEL'] = df['REVEL'].fillna(0)

column_names2=["Transcript ID","LoF"]
df1 = pd.read_csv(f'{TMP_DIR}/splitted_output2.csv',sep=' ', names=column_names2)
df = pd.merge(df, df1, on="Transcript ID")
column_names3=["Locus (hg38)","Ref","Alt","Pangolin scores"]
df2 = pd.read_csv(f'{TMP_DIR}/splitted_output3.vcf',sep=' ', names=column_names3)
df = pd.merge(df, df2, on="Locus (hg38)")
df = df.rename(columns={'Ref_x': 'Ref', 'Alt_x': 'Alt'})
del df["Ref_y"]	
del df["Alt_y"]

df = df.loc[(df['Mane Select']!=".")].reset_index(drop=True)

new_column_8 = df["Pangolin scores"].str.split("|", n=4, expand=True)
df.insert(29,'Pangolin Splice gain (pos:score)',new_column_8[1])
new_column_9 = df["Pangolin scores"].str.split("|", n=4, expand=True)
df.insert(29,'Pangolin Splice loss (pos:score)',new_column_9[2])
new_column_10 = df["Pangolin Splice gain (pos:score)"].str.split(":", n=1, expand=True)
df.insert(29,'Pangolin Splice gain score',new_column_10[1])
new_column_11 = df["Pangolin Splice loss (pos:score)"].str.split(":", n=1, expand=True)
df.insert(29,'Pangolin Splice loss score',new_column_11[1])

df['Pangolin Splice gain score'] = pd.to_numeric(df['Pangolin Splice gain score'], errors='coerce') 
df['Pangolin Splice loss score'] = pd.to_numeric(df['Pangolin Splice loss score'], errors='coerce')

df_1 = df[(df['Variant Type']=="intron_variant")] 
df_1 = df_1.reset_index(drop=True)
df_2 = df[(df['Variant Type']!="intron_variant") & (df['Variant Type']!="frameshift_variant") & (df['Variant Type']!="stop_gained")] 
df_2 = df_2.reset_index(drop=True) 
df_3 = df[(df['Variant Type']=="frameshift_variant") | (df['Variant Type']=="stop_gained")]
df_3 = df_3.reset_index(drop=True) 

r = []
for i in range(len(df_3)):
    if (df_3.loc[i, "EXON"] != 1 and df_3.loc[i, "EXON"] != df_3.loc[i, "EXON TOT"]):
        r.append("Candidate")
    else:
        r.append("Other")  
df_3['ASO'] = r

tier = []
for i in range(len(df_3)):
    tier.append("Tier 2")
df_3['Tier'] = tier
df_3.to_csv(f'{TMP_DIR}/frameshift_stopgain_variants.csv', sep='\t', index=False)

r1 = []
for i in range(len(df_2)):
    if (df_2.loc[i, "Variant Type"] == "start_lost" or 
       (df_2.loc[i, "Variant Type"] == "missense_variant" and  
       (df_2.loc[i, "REVEL"] >= 0.5 or df_2.loc[i, "AlphaMissense"] >= 0.7))):
        r1.append("SEVERE")
    elif (df_2.loc[i, "Variant Type"] == "synonymous_variant" and df_2.loc[i, "REVEL"] < 0.5) or (df_2.loc[i, "Variant Type"] == "missense_variant" and (df_2.loc[i, "REVEL"] < 0.5 or df_2.loc[i, "AlphaMissense"] < 0.7)):
        r1.append("NO TO LITTLE")
    elif (df_2.loc[i, "Variant Type"] == "synonymous_variant" and df_2.loc[i, "REVEL"] >= 0.5):
        r1.append("MODERATE")
    else:
        r1.append(".")  
df_2['Damage to coding function'] = r1  

r2 = []
for i in range(len(df_2)):
    if (df_2.loc[i, "SpliceAI DL score"] >= 0.1 or df_2.loc[i, "SpliceAI AL score"] >= 0.1):
        if (df_2.loc[i, "MaxEntScan Alt"] < df_2.loc[i, "MaxEntScan Ref"] and 
           (df_2.loc[i, "MaxEntScan Alt"] < 2 or 
           df_2.loc[i, "MaxEntScan Alt"] < 0.3 * df_2.loc[i, "MaxEntScan Ref"])):
            r2.append("SEVERE")
        elif (df_2.loc[i, "MaxEntScan Alt"] < df_2.loc[i, "MaxEntScan Ref"] and 
             (df_2.loc[i, "MaxEntScan Alt"] >= 2 or 
             df_2.loc[i, "MaxEntScan Alt"] >= 0.3 * df_2.loc[i, "MaxEntScan Ref"])):
            r2.append("MODERATE")
        else:
            r2.append("NO TO LITTLE")
    else:
        r2.append(".")
df_2['Damage to canonical splicing'] = r2  

r3 = []
for i in range(len(df_2)):
    if (df_2.loc[i, "SpliceAI DG score"] >= 0.1 or 
        df_2.loc[i, "SpliceAI AG score"] >= 0.1 or 
        df_2.loc[i, "MaxEntScan Alt"] >= 2 or 
        (df_2.loc[i, "Pangolin Splice gain score"] >= 0.2 and 
         df_2.loc[i, "Pangolin Splice loss score"] <= 0.2)):
        r3.append("GAIN") 
    elif ((df_2.loc[i, "SpliceAI DL score"] >= 0.1 or 
           df_2.loc[i, "SpliceAI AL score"] >= 0.1) and 
          (df_2.loc[i, "SpliceAI DG score"] < 0.1 or 
           df_2.loc[i, "SpliceAI AG score"] < 0.1) or 
          (df_2.loc[i, "Pangolin Splice gain score"] <= 0.2 and 
           df_2.loc[i, "Pangolin Splice loss score"] >= 0.2)):
        r3.append("SKIPPING OR RETENTION")   
    else:
        r3.append(".")
df_2['Mis-splicing type'] = r3

aso = []
for i in range(len(df_2)):
    if (df_2.loc[i, "Damage to coding function"] == "NO TO LITTLE" and 
        df_2.loc[i, "Damage to canonical splicing"] == "NO TO LITTLE" and 
        df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("PROBABLY")  # High likelihood of ASO amenability
    elif (df_2.loc[i, "Damage to coding function"] == "NO TO LITTLE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "MODERATE" and 
          df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("POSSIBLY")  # Moderate likelihood of ASO amenability
    elif (df_2.loc[i, "Damage to coding function"] == "NO TO LITTLE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "NO TO LITTLE" and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "NO TO LITTLE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "MODERATE" and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "MODERATE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "NO TO LITTLE" and 
          df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "MODERATE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "MODERATE" and 
          df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "MODERATE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "NO TO LITTLE" and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "MODERATE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "MODERATE" and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "NO TO LITTLE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "." and 
          df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("PROBABLY")
    elif (df_2.loc[i, "Damage to coding function"] == "." and 
          df_2.loc[i, "Damage to canonical splicing"] == "NO TO LITTLE" and 
          df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("PROBABLY")
    elif (df_2.loc[i, "Damage to coding function"] == "NO TO LITTLE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "." and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "." and 
          df_2.loc[i, "Damage to canonical splicing"] == "NO TO LITTLE" and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "MODERATE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "." and 
          df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "." and 
          df_2.loc[i, "Damage to canonical splicing"] == "MODERATE" and 
          df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "MODERATE" and 
          df_2.loc[i, "Damage to canonical splicing"] == "." and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "." and 
          df_2.loc[i, "Damage to canonical splicing"] == "MODERATE" and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("POSSIBLY")
    elif (df_2.loc[i, "Damage to coding function"] == "." and 
          df_2.loc[i, "Damage to canonical splicing"] == "." and 
          df_2.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso.append("AMENABLE")
    elif (df_2.loc[i, "Damage to coding function"] == "." and 
          df_2.loc[i, "Damage to canonical splicing"] == "." and 
          df_2.loc[i, "Mis-splicing type"] == "GAIN"):
        aso.append("AMENABLE")
    else:
        aso.append(".")
df_2['ASO-amenability'] = aso

aso1 = []
for i in range(len(df_2)):
    if df_2.loc[i, "ASO-amenability"] == "POSSIBLY" or df_2.loc[i, "ASO-amenability"] == "PROBABLY":
        aso1.append("Candidate") 
    else:
        aso1.append("Other")
df_2['ASO'] = aso1

tier = []
for i in range(len(df_2)):
    if (df_2.loc[i, "Variant Type"] == "synonymous_variant" and df_2.loc[i, "REVEL"] < 0.5 or 
        df_2.loc[i, "Variant Type"] != "missense_variant" and df_2.loc[i, "REVEL"] < 0.5):
        tier.append("Tier 1")
    elif (df_2.loc[i, "Variant Type"] == "synonymous_variant" and 
          (df_2.loc[i, "REVEL"] >= 0.5 or df_2.loc[i, "AlphaMissense"] >= 0.5)):
        tier.append("Tier 3")
    elif (df_2.loc[i, "Variant Type"] == "missense_variant" and 
          (df_2.loc[i, "REVEL"] < 0.5 or df_2.loc[i, "AlphaMissense"] < 0.5)):
        tier.append("Tier 4")
    else:
        tier.append("Tier 5")
df_2['Tier'] = tier

df_2.to_csv(f'{TMP_DIR}/non_intron_variants.csv', sep='\t', index=False)

r4 = []
for i in range(len(df_1)):
    if (df_1.loc[i, "SpliceAI DL score"] >= 0.1 or df_1.loc[i, "SpliceAI AL score"] >= 0.1):
        if (df_1.loc[i, "MaxEntScan Alt"] < df_1.loc[i, "MaxEntScan Ref"] and 
            (df_1.loc[i, "MaxEntScan Alt"] < 2 or 
             df_1.loc[i, "MaxEntScan Alt"] < 0.3 * df_1.loc[i, "MaxEntScan Ref"])):
            r4.append("SEVERE")
        elif (df_1.loc[i, "MaxEntScan Alt"] < df_1.loc[i, "MaxEntScan Ref"] and 
              (df_1.loc[i, "MaxEntScan Alt"] >= 2 or 
               df_1.loc[i, "MaxEntScan Alt"] >= 0.3 * df_1.loc[i, "MaxEntScan Ref"])):
            r4.append("MODERATE")
        else:
            r4.append("NO TO LITTLE")
    else:
        r4.append(".")
df_1['Damage to canonical splicing'] = r4

r5 = []
for i in range(len(df_1)):
    if (df_1.loc[i, "SpliceAI DG score"] >= 0.1 or df_1.loc[i, "SpliceAI AG score"] >= 0.1 or 
        df_1.loc[i, "MaxEntScan Alt"] >= 2 or 
        (df_1.loc[i, "Pangolin Splice gain score"] >= 0.2 and df_1.loc[i, "Pangolin Splice loss score"] <= 0.2)):
        r5.append("GAIN") 
    elif ((df_1.loc[i, "SpliceAI DL score"] >= 0.1 or df_1.loc[i, "SpliceAI AL score"] >= 0.1) and 
          (df_1.loc[i, "SpliceAI DG score"] < 0.1 or df_1.loc[i, "SpliceAI AG score"] < 0.1) or 
          (df_1.loc[i, "Pangolin Splice gain score"] <= 0.2 and df_1.loc[i, "Pangolin Splice loss score"] >= 0.2)):
        r5.append("SKIPPING OR RETENTION") 
    else:
        r5.append(".")
df_1['Mis-splicing type'] = r5

aso2 = []
for i in range(len(df_1)):
    if (df_1.loc[i, "Damage to canonical splicing"] == "NO TO LITTLE" and 
        df_1.loc[i, "Mis-splicing type"] == "GAIN"):
        aso2.append("PROBABLY")
    elif (df_1.loc[i, "Damage to canonical splicing"] == "NO TO LITTLE" and 
          df_1.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso2.append("POSSIBLY")
    elif (df_1.loc[i, "Damage to canonical splicing"] == "MODERATE" and 
          df_1.loc[i, "Mis-splicing type"] == "GAIN"):
        aso2.append("POSSIBLY")
    elif (df_1.loc[i, "Damage to canonical splicing"] == "MODERATE" and 
          df_1.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso2.append("POSSIBLY")
    elif (df_1.loc[i, "Damage to canonical splicing"] == "." and 
          df_1.loc[i, "Mis-splicing type"] == "SKIPPING OR RETENTION"):
        aso2.append("POSSIBLY")
    elif (df_1.loc[i, "Damage to canonical splicing"] == "." and 
          df_1.loc[i, "Mis-splicing type"] == "GAIN"):
        aso2.append("POSSIBLY")
    else:
        aso2.append(".")
df_1['ASO-amenability'] = aso2

aso3 = []
for i in range(len(df_1)):
    if (df_1.loc[i, "ASO-amenability"] == "POSSIBLY" or df_1.loc[i, "ASO-amenability"] == "PROBABLY"):
        aso3.append("Candidate")
    else:
        aso3.append("Other")
df_1['ASO'] = aso3

tier1 = ["Tier 1"] * len(df_1)
df_1['Tier'] = tier1
df_1.to_csv(f'{TMP_DIR}/intron_variants.csv', sep='\t', index=False)

df_4 = df_1.loc[df_1['ASO'] == "Candidate"].reset_index(drop=True)
df_5 = df_2.loc[df_2['ASO'] == "Candidate"].reset_index(drop=True)
df_6 = df_3.loc[df_3['ASO'] == "Candidate"].reset_index(drop=True)
result = pd.concat([df_4, df_5, df_6])

columns_to_drop = ["Pangolin scores", "Pangolin Splice loss (pos:score)", 
                   "Pangolin Splice gain (pos:score)", "Mane Select", 
                   "INTRON TOT", "ASO-amenability", "ASO", 
                   "Damage to canonical splicing", "Damage to coding function"]
result.drop(columns_to_drop, axis=1, inplace=True)

column_names = list(result.columns.values)
column_names.insert(33, column_names.pop(-1))
result = result[column_names]
pfam = pd.read_csv('full/path/to/pfam.csv', sep='\t') ### add here your full path to pfam.csv
pfam2 = pfam.drop_duplicates()
result2 = pd.merge(result, pfam2, how="left", on=["PFAM_ACCESSION"])
result2.to_csv(f'{TMP_DIR}/candidate_variants.csv', sep='\t', index=False)

(result2["Gene"] + ":" + result2["Transcript ID"]).to_csv(f'{TMP_DIR}/gene_transcript.csv', sep='\t', index=False)
gene_transcript = pd.read_csv(f'{TMP_DIR}/gene_transcript.csv', sep='\t')
new22 = gene_transcript["0"].str.split(":", n=1, expand=True)
gene_transcript.insert(0, 'Gene', new22[0])
gene_transcript.insert(1, 'Transcript ID', new22[1])
del gene_transcript["0"]

gene_transcript.to_csv(f'{TMP_DIR}/gene_transcript2.csv',sep=" ", index=False) 

In [ ]:
#importing the libraries required for the project
import requests
import pandas as pd

def get_exon_coordinates_from_df(gene_transcript_df):
    exon_data = []
    for index, row in gene_transcript_df.iterrows():
        gene_name = row['Gene']
        transcript_id = row['Transcript ID']
        server = "https://rest.ensembl.org"
        ext = f"/lookup/symbol/human/{gene_name}?expand=1"
        headers = {"Content-Type": "application/json"}
        response = requests.get(server + ext, headers=headers)
        if not response.ok:
            response.raise_for_status()
        data = response.json()
        if 'Transcript' in data:
            for transcript in data['Transcript']:
                if transcript['id'] == transcript_id:
                    i = 1
                    for exon in transcript['Exon']:
                        start = exon['start']
                        end = exon['end']
                        length = (end - start) + 1
                        exon_data.append({
                            'Gene': gene_name,
                            'Transcript ID': transcript['id'],
                            'EXON': i,
                            'Exon Start': start,
                            'Exon End': end,
                            'Exon Length': length,
                        })
                        i += 1
        else:
            print(f"No transcript found for gene: {gene_name}")
    
    exon_df = pd.DataFrame(exon_data)
    return exon_df

gene_transcript_df = pd.read_csv('/full/path/to/gene_transcript2.csv', sep=' ') # add here your full path to gene_transcript2.csv
exon_df = get_exon_coordinates_from_df(gene_transcript_df)
exon_df.to_csv(f'{TMP_DIR}/gene_tra_exon.csv',sep=' ', index=False)

In [ ]:
ex = pd.read_csv(f'{TMP_DIR}/gene_tra_exon.csv', sep=' ')
ex2 = ex.drop_duplicates()

ex2.to_csv(f'{TMP_DIR}/gene_transcript_exon_nodup.csv', sep=' ', index=False)
ex2['mRNA Length'] = ex2['Exon Length'].groupby(ex2['Gene']).transform('sum')
ex2.to_csv(f'{TMP_DIR}/gene_transcript_exon_nodup_mrnatot.csv', sep=' ', index=False)
ex3 = pd.read_csv(f'{TMP_DIR}/gene_transcript_exon_nodup_mrnatot.csv', sep=' ')

result3 = pd.merge(result2, ex2, how="left", on=["Gene", "Transcript ID", "EXON"])
result3.to_csv(f'{TMP_DIR}/candidate_variants2.csv', sep=' ', index=False)

In [ ]:
sk_ex = []
len_ex = []
i = 0

while i < len(result3):
    exon_skipping_result = "No"
    exon_sum = 0
    j = 0
    
    while j < len(ex3):
        if (result3.loc[i, "Gene"] == ex3.loc[j, "Gene"] and 
            result3.loc[i, "Transcript ID"] == ex3.loc[j, "Transcript ID"] and 
            result3.loc[i, "EXON"] == ex3.loc[j, "EXON"]):
            max_ex = math.ceil(min(10/100 * result3.loc[i, "EXON TOT"], result3.loc[i, "EXON TOT"] - result3.loc[i, "EXON"]))
            y = 0
            
            while y < max_ex and j + y < len(ex3):
                exon_sum += ex3.loc[j + y, "Exon Length"]
                if (ex3.loc[j, "mRNA Length"] - exon_sum) % 3 == 0:
                    exon_skipping_result = str(y+1) + " exons"
                    len_ex.append(exon_sum)
                    break
                y += 1
            
            if exon_skipping_result != "No":
                break
        
        j += 1
    
    sk_ex.append(exon_skipping_result)
    
    if exon_skipping_result == "No":
        len_ex.append(0)
    
    i += 1

result3['Exon Skipping'] = sk_ex
result3['Exon Skipping Length'] = len_ex
result4 = result3.drop_duplicates()
result4.to_csv(f'{TMP_DIR}/candidate_variants3.csv', sep=' ', index=False)

In [ ]:
res_f = pd.read_csv(f'{TMP_DIR}/candidate_variants3.csv', sep=' ')
df_p = pd.read_csv('full/path/to/poisonex_haploinsuf.csv',sep=',') ### add here your full path to poisonex_haploinsuf.csv

dfp = pd.merge(res_f, df_p, how="left", on="Gene")

del dfp["uORF"]
del dfp["NAT"]
del dfp["Pathogenic (ClinVar)"]
del dfp["Contains Known Splice Variants"]
del dfp["pLI"]
del dfp["Unnamed: 0"]

# Final Output
dfp.to_csv(f'{TMP_DIR}/CANDIDATE_VARIANTS_POSION.csv', sep='\t', index=False) 